In [ ]:
# Importer les fonctions nécessaires depuis le package src
from src import load_data, vectorize_text, train_model, save_model

# Charger les données prétraitées
df = load_data('../data/processed_data.csv')

# Vectoriser les tweets en utilisant TF-IDF
X, vectorizer = vectorize_text(df['cleaned_tweet'], method='tfidf', max_features=1000)

# Sauvegarder le vectoriseur pour une utilisation ultérieure
save_model(vectorizer, '../models/tfidf_vectorizer.pkl')

# Définir les étiquettes
y = df['label']

# Séparer les données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner les modèles
model_types = ['logistic', 'nb', 'svm', 'rf']
trained_models = {}

for model_type in model_types:
    print(f"Entraînement du modèle: {model_type}")
    model = train_model(X_train, y_train, model_type=model_type)
    trained_models[model_type] = model
    # Sauvegarder chaque modèle
    save_model(model, f'../models/{model_type}_model.pkl')


In [1]:
# Importations nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [2]:
# Charger le DataFrame à partir du fichier pickle
df = pd.read_pickle('df_data_cleaned.pkl')

# Afficher les premières lignes pour vérifier que les données sont chargées
df.head()

,tweet,label,tweet_length,cleaned_tweet
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1,115,awww thats a bummer you shoulda got david ...
1,is upset that he can't update his Facebook by ...,1,111,is upset that he cant update his facebook by t...
2,@Kenichan I dived many times for the ball. Man...,1,89,i dived many times for the ball managed to sa...
3,my whole body feels itchy and like its on fire,0,47,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all....",1,111,no its not behaving at all im mad why am i he...


In [3]:
# Vectorisation des tweets nettoyés en utilisant TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Appliquer TF-IDF sur les tweets nettoyés
X = tfidf_vectorizer.fit_transform(df['cleaned_tweet'])  # Assurez-vous que 'cleaned_tweet' est la bonne colonne
y = df['label']  # Label de la classification

# Séparation des données en ensembles d'entraînement et de test (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# Choix des modèles à entraîner
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": MultinomialNB(),
    "Support Vector Machine (SVM)": SVC(kernel='linear')
}


In [5]:
# Entraînement et évaluation des modèles
for model_name, model in models.items():
    print(f"Entraînement du modèle: {model_name}")
    
    # Entraînement
    model.fit(X_train, y_train)
    
    # Prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)
    
    # Affichage des résultats
    print(f"Évaluation du modèle {model_name} :")
    print(classification_report(y_test, y_pred))
    print("Matrice de confusion :")
    print(confusion_matrix(y_test, y_pred))
    print("\n" + "="*60)

Entraînement du modèle: Logistic Regression
Évaluation du modèle Logistic Regression :
              precision    recall  f1-score   support

           0       0.99      0.45      0.62      1222
           1       0.94      1.00      0.97     10778

    accuracy                           0.94     12000
   macro avg       0.96      0.73      0.79     12000
weighted avg       0.95      0.94      0.93     12000

Matrice de confusion :
[[  552   670]
 [    8 10770]]

Entraînement du modèle: Naive Bayes
Évaluation du modèle Naive Bayes :
              precision    recall  f1-score   support

           0       0.89      0.04      0.07      1222
           1       0.90      1.00      0.95     10778

    accuracy                           0.90     12000
   macro avg       0.89      0.52      0.51     12000
weighted avg       0.90      0.90      0.86     12000

Matrice de confusion :
[[   47  1175]
 [    6 10772]]

Entraînement du modèle: Support Vector Machine (SVM)
Évaluation du modèle Supp

In [6]:
# Optimisation des hyperparamètres avec GridSearchCV (exemple pour le SVM)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

In [ ]:
# Recherche de grille pour SVM
print("Optimisation des hyperparamètres pour le modèle SVM avec GridSearchCV")
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

Optimisation des hyperparamètres pour le modèle SVM avec GridSearchCV


In [ ]:
# Affichage des meilleurs paramètres
print("Meilleurs paramètres trouvés pour SVM :", grid_search.best_params_)

In [ ]:
# Entraînement avec les meilleurs paramètres
best_svm_model = grid_search.best_estimator_


In [ ]:
# Prédictions avec le meilleur modèle
y_pred_best_svm = best_svm_model.predict(X_test)


In [ ]:
# Évaluation du modèle optimisé
print("\nÉvaluation du modèle SVM optimisé :")
print(classification_report(y_test, y_pred_best_svm))
print("Matrice de confusion (SVM optimisé) :")
print(confusion_matrix(y_test, y_pred_best_svm))

### **Description des étapes**

1. **Chargement des données** :
   - Les données prétraitées (tweets nettoyés) sont chargées dans un dataframe `df`. Vous pouvez utiliser `pd.read_csv()` ou tout autre moyen pour charger vos données.
   
2. **Vectorisation avec TF-IDF** :
   - Nous utilisons `TfidfVectorizer` de `sklearn` pour transformer les tweets nettoyés en une matrice de caractéristiques numériques. Cette transformation est importante, car elle permet de convertir les mots en une représentation vectorielle que les modèles de machine learning peuvent comprendre.
   
3. **Séparation des données en ensembles d'entraînement et de test** :
   - Nous divisons les données en un ensemble d'entraînement (80%) et un ensemble de test (20%) en utilisant `train_test_split`. L'objectif est de tester le modèle sur des données qu'il n'a jamais vues pour évaluer sa capacité à généraliser.

4. **Entraînement des modèles** :
   - Trois modèles différents sont entraînés : 
     - **Régression logistique** (`LogisticRegression`), idéale pour les tâches de classification binaire.
     - **Naive Bayes** (`MultinomialNB`), efficace pour les données textuelles.
     - **Support Vector Machine** (`SVC`), une option puissante pour la classification.
   - Pour chaque modèle, nous affichons un **rapport de classification** (`classification_report`) qui inclut des mesures comme la précision, le rappel et la F1-score, ainsi qu'une **matrice de confusion** pour visualiser la performance du modèle.

5. **Optimisation des hyperparamètres** :
   - **GridSearchCV** est utilisé pour effectuer une recherche sur une grille d'hyperparamètres afin d'optimiser le modèle SVM. Cette étape permet de trouver les meilleurs paramètres pour le modèle, comme le paramètre `C`, le type de noyau (`kernel`), et `gamma`.

6. **Sauvegarde du modèle et du vectoriseur** :
   - Après avoir formé le modèle optimisé, nous sauvegardons le modèle entraîné et le vectoriseur TF-IDF à l'aide de `joblib.dump`. Cela permet de réutiliser le modèle dans de futures prédictions sans avoir à l'entraîner à nouveau.

### **Métriques utilisées pour l'évaluation somaire** :

- **Précision** : Proportion des prédictions correctes parmi toutes les prédictions faites.
- **Rappel** : Proportion des vraies positives parmi les éléments qui étaient réellement positifs.
- **F1-Score** : La moyenne harmonique entre la précision et le rappel. Une bonne mesure pour les classes déséquilibrées.
- **Matrice de confusion** : Une matrice qui résume les résultats des prédictions avec les classes prédites versus les classes réelles.

### **Conclusion** :
Le notebook `3_model_training.ipynb` vous guide à travers l'entraînement et l'évaluation de plusieurs modèles de classification pour votre tâche de prédiction basée sur des tweets. Vous pouvez facilement ajouter ou modifier des modèles ou optimiser davantage les hyperparamètres pour améliorer les performances.

In [ ]:
# Sauvegarder le modèle entraîné et le vectoriseur pour réutilisation
import joblib

# Sauvegarder le meilleur modèle
joblib.dump(best_svm_model, 'best_svm_model.pkl')

# Sauvegarder le vectoriseur TF-IDF
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')